# Exploratory Data Analysis (EDA) for the "How to Live" book by Derek Sivers

## 0. Setup

In [1]:
import random
import re
from pathlib import Path

import requests

In [2]:
# file created with markitdown CLI tool and slightly modified by hand
MD_FILE: Path = Path.cwd().parent / ".data" / "how_to_live__sivers.md"

## 1. Load text and split chapters

In [3]:
md_content = MD_FILE.read_text(encoding="utf-8")
cleaned_content = re.sub(r"^\d{1,3}$\n?", "", md_content, flags=re.MULTILINE)

In [4]:
# Create directory name based on the MD file name
dir_name = MD_FILE.stem.lower().replace(" ", "_")
output_dir = Path.cwd().parent / ".data" / dir_name
output_dir.mkdir(exist_ok=True)

# Split content by ## headings
sections = re.split(r"^## ", cleaned_content, flags=re.MULTILINE)

# Process each section (skip the first empty section if exists)
for i, section in enumerate(sections):
    if not section.strip():
        continue

    # Add back the ## prefix for sections after the first
    if i > 0:
        section = "## " + section

    # Join lines within paragraphs while preserving blank line separation
    paragraphs = section.split("\n\n")
    processed_paragraphs = []

    for paragraph in paragraphs:
        # Join lines within each paragraph, removing line breaks but keeping spaces
        joined_paragraph = " ".join(
            line.strip() for line in paragraph.split("\n") if line.strip()
        )
        if joined_paragraph:
            processed_paragraphs.append(joined_paragraph)

    # Rejoin paragraphs with double line breaks
    section = "\n\n".join(processed_paragraphs)

    # Extract the heading name for filename
    lines = section.strip().split("\n")
    if lines and lines[0].startswith("##"):
        heading = lines[0].replace("##", "").strip()
        # Convert to lowercase and replace spaces/special chars with underscores
        filename = re.sub(r"[^\w\s-]", "", heading.lower())
        filename = re.sub(r"[-\s]+", "_", filename)
        filename = f"{filename}.md"

        # Save to file
        file_path = output_dir / filename
        file_path.write_text(section.replace("##", "#"), encoding="utf-8")
        print(f"Created: {file_path}")

print(f"\nAll sections saved to directory: {output_dir}")

Created: /Users/uziel/Development/twenty-seven/.data/how_to_live__sivers/heres_how_to_live_be_independent.md
Created: /Users/uziel/Development/twenty-seven/.data/how_to_live__sivers/heres_how_to_live_commit.md
Created: /Users/uziel/Development/twenty-seven/.data/how_to_live__sivers/heres_how_to_live_fill_your_senses.md
Created: /Users/uziel/Development/twenty-seven/.data/how_to_live__sivers/heres_how_to_live_do_nothing.md
Created: /Users/uziel/Development/twenty-seven/.data/how_to_live__sivers/heres_how_to_live_think_super_long_term.md
Created: /Users/uziel/Development/twenty-seven/.data/how_to_live__sivers/heres_how_to_live_intertwine_with_the_world.md
Created: /Users/uziel/Development/twenty-seven/.data/how_to_live__sivers/heres_how_to_live_make_memories.md
Created: /Users/uziel/Development/twenty-seven/.data/how_to_live__sivers/heres_how_to_live_master_something.md
Created: /Users/uziel/Development/twenty-seven/.data/how_to_live__sivers/heres_how_to_live_let_randomness_rule.md
Creat

## 2. Summarise chapters to create final perspectives

In [5]:
# Configuration
CHAPTERS_DIR = output_dir
OUTPUT_DIR = CHAPTERS_DIR / "summaries"
OUTPUT_DIR.mkdir(exist_ok=True)
SUMMARY_WORD_LIMIT = 256

# LM Studio configuration (adjust endpoint as needed)
LM_STUDIO_ENDPOINT = "http://localhost:1234/v1/chat/completions"


def remove_think_tags(text):
    """Remove <think>...</think> tags and their content from the text."""
    return re.sub(r"<think>[\s\S]*?</think>", "", text, flags=re.IGNORECASE)


def summarize_chapter(content, word_limit):
    """Summarize a chapter using LM Studio API"""
    prompt = f"""
    Please summarize the following text which represents a philosophy of life perspective on "how to live".
    The summary should be written as an instruction to another person, explaining how to act and think about life and decisions following this particular philosphy of life.
    To that effect, you can use expressions such as "You think...", "You believe...", "You like...", "You love...", "You prefer...", and any other similar ones, to convey the advice clearly and directly.
    
    Keep the summary to a maximum of {word_limit} words and focus on the key philosophical insights and practical advice.

    Text to summarize:
    {content}

    Return the summary text only, without any additional commentary or formatting.
    """

    payload = {
        "model": "qwen3-30b-a3b",  # Adjust model name as needed
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant that creates concise, insightful summaries of philosophical texts.",
            },
            {"role": "user", "content": prompt},
        ],
        "temperature": 0.3,
        "max_tokens": 2048,
    }

    try:
        response = requests.post(LM_STUDIO_ENDPOINT, json=payload)
        response.raise_for_status()
        result = response.json()
        summary = result["choices"][0]["message"]["content"].strip()
        summary = remove_think_tags(summary)
        return summary
    except Exception as e:
        print(f"Error calling LM Studio API: {e}")
        return None


# Process each markdown file in the chapters directory
for md_file in sorted(list(CHAPTERS_DIR.glob("*.md"))):
    print(f"Processing: {md_file.name}")

    # Read the chapter content
    content = md_file.read_text(encoding="utf-8")

    # Generate summary
    summary = summarize_chapter(content, SUMMARY_WORD_LIMIT)

    if summary:
        # Strip all line breaks and leading/trailing spaces, collapse multiple spaces
        summary_clean = re.sub(
            r"\s+", " ", summary.replace("\n", " ").replace("\r", " ")
        ).strip()

        # Create summary filename
        summary_filename = md_file.stem + "_summary.txt"
        summary_path = OUTPUT_DIR / summary_filename

        # Save summary
        summary_path.write_text(
            summary_clean,
            encoding="utf-8",
        )
        print(f"Created summary: {summary_path.name}")
    else:
        print(f"Failed to generate summary for {md_file.name}")

print(f"\nSummaries completed and saved to: {CHAPTERS_DIR}")

Processing: conclusion.md
Created summary: conclusion_summary.txt
Processing: heres_how_to_live_balance_everything.md
Created summary: heres_how_to_live_balance_everything_summary.txt
Processing: heres_how_to_live_be_a_famous_pioneer.md
Created summary: heres_how_to_live_be_a_famous_pioneer_summary.txt
Processing: heres_how_to_live_be_independent.md
Created summary: heres_how_to_live_be_independent_summary.txt
Processing: heres_how_to_live_chase_the_future.md
Created summary: heres_how_to_live_chase_the_future_summary.txt
Processing: heres_how_to_live_commit.md
Created summary: heres_how_to_live_commit_summary.txt
Processing: heres_how_to_live_create.md
Created summary: heres_how_to_live_create_summary.txt
Processing: heres_how_to_live_do_nothing.md
Created summary: heres_how_to_live_do_nothing_summary.txt
Processing: heres_how_to_live_do_whatever_you_want_now.md
Created summary: heres_how_to_live_do_whatever_you_want_now_summary.txt
Processing: heres_how_to_live_dont_die.md
Created su

## 3. Test answers given different philosophies

In [7]:
def get_random_philosophy():
    """Get a random philosophy summary from the generated files"""
    summary_files = list((OUTPUT_DIR).glob("*_summary.txt"))
    if not summary_files:
        return None, None

    selected_file = random.choice(summary_files)
    philosophy = selected_file.read_text(encoding="utf-8").strip()
    philosophy_name = (
        selected_file.stem.replace("_summary", "").replace("_", " ").title()
    )

    return philosophy_name, philosophy


def answer_with_philosophy(question, philosophy_name, philosophy):
    """Answer a question using a specific philosophy"""
    prompt = f"""
    You are answering a life question from the perspective of this specific philosophy:

    Philosophy:
    {philosophy_name}
    
    Perspective:
    {philosophy}

    Question:
    {question}

    Answer the question by embodying this philosophy completely.
    Write as if you truly believe in this perspective and are giving advice based on these principles.
    Be specific, concise and practical in your guidance. Avoid unnecessary verbosity.

    Answer:
    """

    payload = {
        "model": "qwen3-30b-a3b",
        "messages": [
            {
                "role": "system",
                "content": "You are a wise advisor who answers questions by embodying specific life philosophies completely.",
            },
            {"role": "user", "content": prompt},
        ],
        "temperature": 0.2,
        "max_tokens": 2048,
    }

    try:
        response = requests.post(LM_STUDIO_ENDPOINT, json=payload)
        response.raise_for_status()
        result = response.json()
        answer = result["choices"][0]["message"]["content"].strip()
        answer = remove_think_tags(answer)
        return answer
    except Exception as e:
        print(f"Error calling LM Studio API: {e}")
        return None


# Example question - a common life dilemma
question = """
I'm 28 years old and work as a software engineer at a large tech company.
The pay is excellent and the job is stable, but I find the work increasingly meaningless.
I've always been passionate about photography and have been building a portfolio in my spare time.
Some of my work has been featured in local galleries. 

I'm considering leaving my job to pursue photography full-time, but I'm scared about the financial uncertainty.
I have some savings, but not enough to last more than a year without income.
My family thinks I'm crazy to consider giving up a "good job" for something so uncertain.

Should I take the leap and pursue my passion, or stay in my secure but unfulfilling job?
"""

# Get a random philosophy and answer the question
philosophy_name, philosophy = get_random_philosophy()

if philosophy_name and philosophy:
    print("=" * 80)
    print("PHILOSOPHICAL LIFE ADVICE SYSTEM")
    print("=" * 80)
    print(f"\nQUESTION:\n{question}")
    print(f"\nSELECTED PHILOSOPHY: {philosophy_name}")

    answer = answer_with_philosophy(question, philosophy_name, philosophy)

    if answer:
        print(f"\nADVICE FROM '{philosophy_name}' PERSPECTIVE:")
        print("-" * 50)
        print(answer)
    else:
        print("\nFailed to generate answer")
else:
    print("No philosophy summaries found. Run section 2 first.")

PHILOSOPHICAL LIFE ADVICE SYSTEM

QUESTION:

I'm 28 years old and work as a software engineer at a large tech company.
The pay is excellent and the job is stable, but I find the work increasingly meaningless.
I've always been passionate about photography and have been building a portfolio in my spare time.
Some of my work has been featured in local galleries. 

I'm considering leaving my job to pursue photography full-time, but I'm scared about the financial uncertainty.
I have some savings, but not enough to last more than a year without income.
My family thinks I'm crazy to consider giving up a "good job" for something so uncertain.

Should I take the leap and pursue my passion, or stay in my secure but unfulfilling job?


SELECTED PHILOSOPHY: Heres How To Live Balance Everything

ADVICE FROM 'Heres How To Live Balance Everything' PERSPECTIVE:
--------------------------------------------------


Balance is not about choosing one path over another—it’s about weaving them together with